In [1]:
"""
- simpy에서 세팅한 시뮬레이션 월드는 프로세스를 중심으로 구현됩니다. 
- 동시에 여러 개의 프로세스를 돌릴 수 있는데(예를 들어 프로세스=차, 여러 개의 차가 드라이빙되고 있음)
- 이 때, 프로세스 간의 인터랙션을 어떻게 구현할 수 있는가? 
    - 일단 여기서는 interrupting/ waiting for a process 라고 구분해서 말하고 있음 

- 프로세스와 subprocess로 구분해서 볼 수도 있을 듯. 
"""
import simpy 
import numpy as np 

## 물론 이 아래 부분을 클래스로 구현을 해도 좋지만 일단은 이해를 위해서 다 함수로 표현함 
def subsubprocess(env):
    ## process의 개별 activity는 subprocess로 구성되어 있습니다. 
    print('        subsubprocess start at {:6.2f}'.format(env.now))
    for i in range(0, 2):
        execution_time = np.random.triangular(left=1, right=2, mode=1)
        yield env.timeout(execution_time)
    print('        subsubprocess over  at {:6.2f}'.format(env.now))
def subprocess(env):
    ## process의 개별 activity는 subprocess로 구성되어 있습니다. 
    print('    subprocess start at {:6.2f}'.format(env.now))
    for i in range(0, 2):
        yield env.process(subsubprocess(env))
    print('    subprocess over  at {:6.2f}'.format(env.now))
    
def process(env, activity_lst):
    while True:
        for act in activity_lst:
            print("start {} at {:6.2f}".format(act, env.now))
            execution_time = np.random.triangular(left=3, right=10, mode=6)
            ## 모든 activity는 subprocess라고 생각한다.
            ## subprocess(env)가 종료되어야 다음 스텝으로 넘어감
            ## 즉 일종의 waiting for other process를 구현했다고 보면 됨 
            yield env.process(subprocess(env))
            ##############
            print("end   {} at {:6.2f}".format(act, env.now))
            transfer_time = np.random.triangular(left=1, right=3, mode=2)
            yield env.timeout(transfer_time)
        print('process instance ends')
        print('#'*30)
        return None
###########
env = simpy.Environment()
process1 = process(env, ["act_{}".format(i) for i in range(0, 3)])
env.process(process1)
env.run(50)

start act_0 at   0.00
    subprocess start at   0.00
        subsubprocess start at   0.00
        subsubprocess over  at   2.77
        subsubprocess start at   2.77
        subsubprocess over  at   5.70
    subprocess over  at   5.70
end   act_0 at   5.70
start act_1 at   7.91
    subprocess start at   7.91
        subsubprocess start at   7.91
        subsubprocess over  at  10.34
        subsubprocess start at  10.34
        subsubprocess over  at  12.78
    subprocess over  at  12.78
end   act_1 at  12.78
start act_2 at  15.53
    subprocess start at  15.53
        subsubprocess start at  15.53
        subsubprocess over  at  18.20
        subsubprocess start at  18.20
        subsubprocess over  at  20.39
    subprocess over  at  20.39
end   act_2 at  20.39
process instance ends
##############################


In [59]:
import simpy 
import numpy as np 

## 현재 env에 있는 모든 process를 쭉 접근할 수 있는 방법이 없음.
## 따라서, 매번 프로세스를 따로 리스트의 형태로 저장해주는 것이 필요함. 
current_ps = []

def clock(env, i, tick):
    ## generator에 interrupt 가 발생했을 때 종료하는 조건을 넣어주어야 함 
    while True:
        try: 
            yield env.timeout(tick)
            print('clock {} ticks at {}'.format(i, env.now))
        except simpy.Interrupt:
            print("## the clock {} was interrupted".format(i))
            return None
            
def stop_any_process(env):
    ## 2초마다 한번씩 현재 process 중 아무거나 종료시키는 generator
    ## 남아있는 clock이 없을때의 조건도 만들어줌. 
    while True:
        try:
            yield env.timeout(2)
            r = np.random.randint(0, len(current_ps))
            current_ps[r].interrupt()
            current_ps.remove(current_ps[r])
        except:
            print("#"*20)
            print("all process was interrupted at {}".format(env.now))
            return None
        
## environment setting
env = simpy.Environment()

## 6 개의 중간에 멈출 수 있는 clock을 만들어서 집어넣음
for i in range(0, 5):
    p = env.process(clock(env, i, 2))
    ## 새롭게 만들어진 프로세스에 대해서 외부에서 접근 방법이 없으므로, 따로 저장해두어야 함
    current_ps.append(p)

## 2초마다 process를 멈추는 generator도 넘겨줌
env.process(stop_any_process(env))

env.run(until=20)

clock 0 ticks at 2
clock 1 ticks at 2
clock 2 ticks at 2
clock 3 ticks at 2
clock 4 ticks at 2
## the clock 3 was interrupted
clock 0 ticks at 4
clock 1 ticks at 4
clock 2 ticks at 4
clock 4 ticks at 4
## the clock 0 was interrupted
clock 1 ticks at 6
clock 2 ticks at 6
clock 4 ticks at 6
## the clock 4 was interrupted
clock 1 ticks at 8
clock 2 ticks at 8
## the clock 1 was interrupted
clock 2 ticks at 10
## the clock 2 was interrupted
####################
all process was interrupted at 12


In [147]:
import simpy 
import numpy as np 

def Student(env, num, library, arrive_time):
    yield env.timeout(arrive_time)
    print("student {} arrived library at {:6.2f}".format(num, env.now))
    ## 아래와 같은 형태로 쓰면 자동으로 get, release가 된다.
    ## 단, 다른 형태로 쓸 경우에는 req = library.request(), library.release(req) 로 해주어야 함. 
    waiting_time = env.now
    with library.request() as req:
        yield req ## resource를 사용할 수 있으면 이 다음 부분이 수행됨
        waiting_time = env.now - waiting_time
        if waiting_time !=0:
            print("student {} is waiting  during {:6.2f}".format(num, waiting_time))
        study_time = np.random.triangular(left=5, right=10, mode=8)
        print("student {} start to  study at {:6.2f}".format(num, env.now))
        if library.capacity == library.count:
            print("#### library full at  {:6.2f} ####".format(env.now))
        yield env.timeout(study_time)
        print("student {} end   to  study at {:6.2f}".format(num, env.now))
        print("#### library seat available at {:6.2f} ####".format(env.now))
        
env = simpy.Environment()
library = simpy.Resource(env, capacity=2)

for i in range(0, 5):
    arrive_time = np.random.triangular(left=1, right=8, mode=3)
    stu = Student(env, i, library, arrive_time)
    print(stu)
    env.process(stu)

env.run(until=50)

<generator object Student at 0x10fa1a6d0>
<generator object Student at 0x10f9f4b48>
<generator object Student at 0x10f9f4308>
<generator object Student at 0x10f9f4d58>
<generator object Student at 0x10f9f4d00>
student 3 arrived library at   2.44
student 3 start to  study at   2.44
student 4 arrived library at   3.49
student 4 start to  study at   3.49
#### library full at    3.49 ####
student 1 arrived library at   3.71
student 2 arrived library at   3.93
student 0 arrived library at   4.11
student 3 end   to  study at  10.67
#### library seat available at  10.67 ####
student 1 is waiting  during   6.96
student 1 start to  study at  10.67
#### library full at   10.67 ####
student 4 end   to  study at  11.17
#### library seat available at  11.17 ####
student 2 is waiting  during   7.24
student 2 start to  study at  11.17
#### library full at   11.17 ####
student 1 end   to  study at  17.27
#### library seat available at  17.27 ####
student 0 is waiting  during  13.17
student 0 start to 

In [148]:
import simpy 
import numpy as np 

def Student(env, num, library, arrive_time):
    yield env.timeout(arrive_time)
    print("student {} arrived library at {:6.2f}".format(num, env.now))
    ## 아래와 같은 형태로 쓰면 자동으로 get, release가 된다.
    ## 단, 다른 형태로 쓸 경우에는 req = library.request(), library.release(req) 로 해주어야 함. 
    waiting_time = env.now
    with library.request() as req:
        if len(library.queue)!=0:
            print("{} student are already waiting for".format(len(library.queue)))
            print(library.users)
        yield req ## resource를 사용할 수 있으면 이 다음 부분이 수행됨
        waiting_time = env.now - waiting_time
        if waiting_time !=0:
            print("student {} is waiting  during {:6.2f}".format(num, waiting_time))
        study_time = np.random.triangular(left=5, right=10, mode=8)
        print("student {} start to  study at {:6.2f}".format(num, env.now))
        if library.capacity == library.count:
            print("#### library full at  {:6.2f} ####".format(env.now))
        yield env.timeout(study_time)
        print("student {} end   to  study at {:6.2f}".format(num, env.now))
        print("#### library seat available at {:6.2f} ####".format(env.now))
        
env = simpy.Environment()
library = simpy.Resource(env, capacity=2)

for i in range(0, 5):
    arrive_time = np.random.triangular(left=1, right=8, mode=3)
    stu = Student(env, i, library, arrive_time)
    print(stu)
    env.process(stu)

env.run(until=50)

<generator object Student at 0x10fa1a410>
<generator object Student at 0x10f9f4d00>
<generator object Student at 0x10fa1ab48>
<generator object Student at 0x10fa1a3b8>
<generator object Student at 0x10fa1a888>
student 4 arrived library at   1.25
student 4 start to  study at   1.25
student 3 arrived library at   2.92
student 3 start to  study at   2.92
#### library full at    2.92 ####
student 2 arrived library at   2.96
1 student are already waiting for
[<Request() object at 0x10fa10e48>, <Request() object at 0x10fa10fd0>]
student 0 arrived library at   4.45
2 student are already waiting for
[<Request() object at 0x10fa10e48>, <Request() object at 0x10fa10fd0>]
student 1 arrived library at   4.54
3 student are already waiting for
[<Request() object at 0x10fa10e48>, <Request() object at 0x10fa10fd0>]
student 4 end   to  study at   8.76
#### library seat available at   8.76 ####
student 2 is waiting  during   5.80
student 2 start to  study at   8.76
#### library full at    8.76 ####
stud

In [145]:
def aaa():
    i = 1
    while True:
        yield i 
        i+=1
a = aaa()

In [146]:
print(a)

<generator object aaa at 0x10fa1a570>


In [204]:
import numpy as np 

np.random.exponential(10)

4.692680899768591

In [ ]:
import simpy 
import numpy as np 

def customer(env, name, counter, time_in_bank):
    """
    Customer arrives ==> served ==> leaves.
    """
    arrive_time = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        ## 아래처럼 
        results = yield req | env.timeout(patience)

        wait_time = env.now - arrive_time

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait_time))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = np.random.exponential(interval)
        yield env.timeout(t)

In [223]:
import simpy 
import numpy as np 

def customer(env, name, counter, mean_service_time):
    ## counter: 사용하는 리소스 
    ## mean_service_time: 서비스 시간 평균 
    arrive_time = env.now
    print('%7.4f %s: Here I am' % (arrive_time, name))
    
    with counter.request() as req:
        yield req 
        wait_time = env.now - arrive_time
        print('%7.4f %s: Waited %6.3f' % (env.now, name, wait_time))
        service_time = np.random.exponential(mean_service_time)
        yield env.timeout(service_time)
        print('%7.4f %s: Finished' % (env.now, name))

def source(env, customer_n, interval, counter):
    ## exponential time 마다 customer를 추가해줍니다
    for i in range(customer_n):
        c = customer(env, 'Customer%02d' % i, counter, mean_service_time=5.0)
        env.process(c)
        t = np.random.exponential(interval)
        yield env.timeout(t)
        
np.random.seed(42)
env = simpy.Environment()

## 우선 counter generator를 만들어주고 
counter = simpy.Resource(env, capacity=1)
bank = source(env, 5, 3.0, counter)

env.process(bank)
env.run(until=90)


 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 1.4078 Customer01: Here I am
 5.3580 Customer02: Here I am
 8.0969 Customer03: Here I am
 8.6057 Customer04: Here I am
15.0506 Customer00: Finished
15.0506 Customer01: Waited 13.643
15.3498 Customer01: Finished
15.3498 Customer02: Waited  9.992
25.4060 Customer02: Finished
25.4060 Customer03: Waited 17.309
30.0014 Customer03: Finished
30.0014 Customer04: Waited 21.396
36.1576 Customer04: Finished


In [239]:
import simpy 
import numpy as np 

def customer(env, name, counter, mean_service_time):
    ## counter: 사용하는 리소스 
    ## mean_service_time: 서비스 시간 평균 
    arrive_time = env.now
    patience_time = np.random.uniform(2, 5)
    print('%7.4f %s: Here I am' % (arrive_time, name))
    
    with counter.request() as req:
        ## | 로 묶어주면 or  종료조건
        ## & 로 묶어주면 and 종료조건으로 인식함 
        patience_over = env.timeout(patience_time) 
        result = yield req | patience_over
        wait_time = env.now - arrive_time
        if patience_over in result:
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait_time))
        elif req in result:
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait_time))
            service_time = np.random.exponential(mean_service_time)
            yield env.timeout(service_time)
            print('%7.4f %s: Finished' % (env.now, name))

def source(env, customer_n, interval, counter):
    ## exponential time 마다 customer를 추가해줍니다
    for i in range(customer_n):
        c = customer(env, 'Customer%02d' % i, counter, mean_service_time=8.0)
        env.process(c)
        t = np.random.exponential(interval)
        yield env.timeout(t)
        
np.random.seed(42)
env = simpy.Environment()

## 우선 counter generator를 만들어주고 
counter = simpy.Resource(env, capacity=1)
bank = source(env, 5, 3.0, counter)

env.process(bank)
env.run(until=90)

 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 1.4078 Customer01: Here I am
 3.8759 Customer01: RENEGED after  2.468
 4.1466 Customer02: Here I am
 4.6554 Customer03: Here I am
 6.3209 Customer02: RENEGED after  2.174
 8.4588 Customer03: RENEGED after  3.803
10.5340 Customer00: Finished
10.6891 Customer04: Here I am
10.6891 Customer04: Waited  0.000
38.7176 Customer04: Finished


In [ ]:
import simpy 
import numpy as np 


def customer_arrivals(env, theater):
    """Create new *moviegoers* until the sim time reaches 120."""
    while True:
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))

In [304]:
import simpy 
import numpy as np

## 세차머신
class CarWashMachine(object):
    ## 리소스는 아래처럼 클래스로 만들어서 처리해주는 것이 바람직할 것으로 생각됨 
    def __init__(self, env, capacity):
        self.env = env
        self.machine = simpy.Resource(self.env, capacity = capacity)
        self.users = [] ## 현재 사용중인 프로세스
        self.queue = [] ## 현재 대기열에 있는 프로세스 
    def wash(self, car_name):
        ## 아래처럼 resource 에서 이뤄지는 부분은 여기에서 작성하는 편이 더 좋을 수도 있음.
        print("{:6.2f} users: {}, queue: {}".format(env.now, self.users, self.queue))
        waiting_time = self.env.now
        with self.machine.request() as req:
            ## 바로 resource available 할 때 
            if self.machine.count < self.machine.capacity:
                self.users.append(car_name)
                yield req
            ## resource available 하지 않을 때 
            else:
                self.queue.append(car_name)
                yield req
                self.queue.remove(car_name)
                self.users.append(car_name)
            waiting_time = self.env.now - waiting_time
            if waiting_time!=0:
                print("{:6.2f} {} waited {:6.2f}".format(self.env.now, car_name, waiting_time))
            print("{:6.2f} {} wash start".format(self.env.now, car_name))    
            wash_time = np.random.exponential(30)
            yield self.env.timeout(wash_time)
            print("{:6.2f} {} wash over".format(self.env.now, car_name))
        self.users.remove(car_name)

## 도착했음을 출력하고, 리소스에 사용요청을 보내고, 사용을 끝내면 메세지츨 출력하고 종료하는 제너레이터 
def car(env, name, car_wash_machine):
    ## 도착하고, resource에 넘겨지고 그 다음을 죽 진행함. 
    print("{:6.2f} {} arrived".format(env.now, name))
    yield env.process(car_wash_machine.wash(name))
    print("{:6.2f} {} leaved".format(env.now, name))

## 랜덤으로 car를 생성해내는 제너레이터 
def source(env, car_n, car_wash_machine):
    for i in range(0, car_n):
        arrival_time = np.random.exponential(3)
        yield env.timeout(arrival_time)
        new_car = car(env, "Car{:2d}".format(i), car_wash_machine)
        env.process(new_car)

np.random.seed(42)
env = simpy.Environment()
cwm1 = CarWashMachine(env, capacity=2)

s = source(env, 10, cwm1)
env.process(s)

env.run(until=100)

  1.41 Car 0 arrived
  1.41 users: [], queue: []
  1.41 Car 0 wash start
 10.44 Car 1 arrived
 10.44 users: ['Car 0'], queue: []
 10.44 Car 1 wash start
 13.18 Car 2 arrived
 13.18 users: ['Car 0', 'Car 1'], queue: []
 13.69 Car 3 arrived
 13.69 users: ['Car 0', 'Car 1'], queue: ['Car 2']
 13.87 Car 4 arrived
 13.87 users: ['Car 0', 'Car 1'], queue: ['Car 2', 'Car 3']
 15.53 Car 1 wash over
 15.53 Car 1 leaved
 15.53 Car 2 waited   2.35
 15.53 Car 2 wash start
 19.90 Car 5 arrived
 19.90 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4']
 23.59 Car 6 arrived
 23.59 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5']
 23.66 Car 7 arrived
 23.66 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5', 'Car 6']
 34.17 Car 8 arrived
 34.17 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5', 'Car 6', 'Car 7']
 39.53 Car 9 arrived
 39.53 users: ['Car 0', 'Car 2'], queue: ['Car 3', 'Car 4', 'Car 5', 'Car 6', 'Car 7', 'Car 8']
 40.91 Car 0 wash over
 40.91 Car 0 leav

In [406]:
import simpy 
import numpy as np 

class Machine(object):
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False 
        
        self.process = self.env.process(self.working(repairman))
        self.env.process(self.break_machine())
        
    def working(self, repairman):
        while True:
            time_per_part = np.random.normal(20, 2)
            while True:
                start = self.env.now
                try:
                    ## 문제없이 부품을 생산한 경우 
                    yield self.env.timeout(time_per_part)
                    break
                except simpy.Interrupt:
                    ## 부품 생산중에 장비가 고장난 경우 
                    ## 이미 조립한 부분은 남기고, 추가로 조립해야 하는 시간만 계산함 
                    time_per_part -= (self.env.now - start)## remain time to part
                    ## 고장 났기 때문에 repairman에게 우선순위 높게 요청한다. 
                    self.env.process(self.repair_machine(repairman))
                    ## 고침 완료
                    ## 고쳐졌으므로 다시 고장 프로세스 가동
                    self.env.process(self.break_machine())
            self.parts_made += 1
            
    def repair_machine(self, repairman):
        with repairman.request(priority=1) as req:
            yield req ## 사용 요청 
            repair_time = np.random.uniform(30, 40)
            yield self.env.timeout(repair_time) ## 고침 
            print("{} is repaired at {:8.2f}".format(self.name, env.now))
            self.broken = False
            
    def break_machine(self):
        ## 랜덤한 시간 뒤에 현재 broken이 아니면 인터럽트를 발생시킴. 
        time_to_failure = np.random.exponential(300) 
        if self.broken == True:
            time_to_failure += np.random.uniform(30, 40)
        yield self.env.timeout(time_to_failure)
        if self.broken == False:
            print("{} is broken   at {:8.2f}".format(self.name, env.now))
            self.broken = True
            self.process.interrupt()
                
def other_jobs(env, repairman):
    ## repairman은 고치는 일 이외에도 우선순위가 낮은 다른 일들을 계속 수행하고 있음. 
    while True:
        do_time = 30.0
        while True:
            with repairman.request(priority=2) as req:
                yield req
                ## repairman의 경우 그냥 Resource가 아닌 PreemptiveResource
                ## 따라서 현재 하고 있는 일보다 priority가 높은 request가 들어왔는데 capacity가 부족할 경우, 
                ## priority가 우선인 것을 먼저 한다. 따라서 그때는 simpy.Interrupt가 발생하게 됨. 
                try:
                    start = env.now
                    yield env.timeout(do_time)
                    do_time = 0
                    break
                except simpy.Interrupt:
                    do_time -= (env.now - start)
        
####################
np.random.seed(42)
print("Machine shop")

env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)

machines = [Machine(env, 'Machine %d' % i, repairman) for i in range(0, 10)]
env.process(other_jobs(env, repairman))
env.run(until=60*5)

print("#"*31)
print('Machine shop results after %s weeks' % 4)
sum_parts = 0
for machine in machines:
    print('%s made %d parts.' % (machine.name, machine.parts_made))
    sum_parts += machine.parts_made
print("#"*31)
print("{} parts produced".format(sum_parts))

Machine shop
Machine 4 is broken   at     6.24
Machine 2 is broken   at    17.95
Machine 4 is repaired at    39.16
Machine 6 is broken   at    60.20
Machine 7 is broken   at    60.78
Machine 2 is repaired at    74.11
Machine 9 is broken   at   103.27
Machine 6 is repaired at   107.42
Machine 7 is repaired at   138.50
Machine 7 is broken   at   144.37
Machine 8 is broken   at   169.66
Machine 9 is repaired at   176.58
Machine 6 is broken   at   194.57
Machine 7 is repaired at   211.60
Machine 8 is repaired at   250.97
Machine 1 is broken   at   273.88
Machine 6 is repaired at   287.89
###############################
Machine shop results after 4 weeks
Machine 0 made 14 parts.
Machine 1 made 14 parts.
Machine 2 made 14 parts.
Machine 3 made 16 parts.
Machine 4 made 14 parts.
Machine 5 made 14 parts.
Machine 6 made 15 parts.
Machine 7 made 14 parts.
Machine 8 made 15 parts.
Machine 9 made 15 parts.
###############################
145 parts produced


In [ ]:
import simpy 
import numpy as np 

class Machine(object):
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False 
        
        # Start "working" and "break_machine" processes for this machine.
        self.process = self.env.process(self.working(repairman))
        self.env.process(self.break_machine())
        
    def working(self, repairman):
        while True:
            time_per_part = np.random.normal(20, 2)
            while True:
                start = self.env.now
                try:
                    ## 문제없이 부품을 생산한 경우 
                    yield self.env.timeout(time_per_part)
                    break
                except simpy.Interrupt:
                    ## 부품 생산중에 장비가 고장난 경우 
                    ## 이미 조립한 부분은 남기고, 추가로 조립해야 하는 시간만 계산함 
                    time_per_part -= (self.env.now - start)## remain time to part
                    ## 고장 났기 때문에 repairman에게 우선순위 높게 요청한다. 
                    with repairman.request(priority=1) as req:
                        yield req ## 사용 요청 
                        repair_time = np.random.uniform(30, 40)
                        yield self.env.timeout(repair_time) ## 고침 
                        #print("{} is repaired at {:8.2f}".format(self.name, env.now))
                    ## 고침 완료
                    self.broken = False
                    ## 고쳐졌으므로 다시 고장 프로세스 가동
                    self.env.process(self.break_machine())
            self.parts_made += 1
    def repair_machine(self, repairman):
        
    def break_machine(self):
        ## 랜덤한 시간 뒤에 현재 broken이 아니면 인터럽트를 발생시킴. 
        time_to_failure = np.random.exponential(300) 
        if self.broken == True:
            time_to_failure += np.random.uniform(30, 40)
        yield self.env.timeout(time_to_failure)
        if self.broken == False:
            #print("{} is broken   at {:8.2f}".format(self.name, env.now))
            self.broken = True
            self.process.interrupt()
                
def other_jobs(env, repairman):
    while True:
        do_time = 30.0
        while True:
            with repairman.request(priority=2) as req:
                ## repairman의 경우 그냥 Resource가 아닌 PreemptiveResource
                ## 따라서 현재 하고 있는 일보다 priority가 높은 request가 들어왔는데 capacity가 부족할 경우, 
                ## priority가 우선인 것을 먼저 한다. 따라서 그때는 simpy.Interrupt가 발생하게 됨. 
                yield req
                try:
                    start = env.now
                    yield env.timeout(do_time)
                    do_time = 0
                    break
                except simpy.Interrupt:
                    do_time -= (env.now - start)
        

np.random.seed(42)
print("Machine shop")

env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)

machines = [Machine(env, 'Machine %d' % i, repairman) for i in range(0, 10)]
env.process(other_jobs(env, repairman))
env.run(until=60*3)
print("#"*31)
print('Machine shop results after %s weeks' % 4)
sum_parts = 0
for machine in machines:
    print('%s made %d parts.' % (machine.name, machine.parts_made))
    sum_parts += machine.parts_made
print("#"*31)
print("{} parts produced".format(sum_parts))

In [408]:
[np.random.poisson(10) for i in range(0, 10)]

[16, 14, 9, 8, 12, 5, 10, 12, 12, 8]

In [411]:
a = {'k_{}'.format(k): "v_{}".format(k) for k in range(0, 10)}
a

{'k_0': 'v_0',
 'k_1': 'v_1',
 'k_2': 'v_2',
 'k_3': 'v_3',
 'k_4': 'v_4',
 'k_5': 'v_5',
 'k_6': 'v_6',
 'k_7': 'v_7',
 'k_8': 'v_8',
 'k_9': 'v_9'}

In [412]:
a.k_0

AttributeError: 'dict' object has no attribute 'k_0'

In [499]:
import simpy 
import numpy as np 

def movie_goer(env, movie, num_tickets, theater):
    ## 영화보러 온 사람 프로세스 
    ## 카운터에 리소스를 요청하고 기다리는 중에 
    ## sold_out이라는 전역 이벤트가 발생하면 renege, 
    ## 그렇찌 않을때는 다른 프로세스를 따름. 
    with theater['counter'].request() as req:
        ## sold_out은 env.event(), 만약 다른 프로세스에서 succeed를 처리하면 아래에서 먼저 실행됨 
        sold_out = theater['sold_out'][movie]
        result = yield req | sold_out
        if sold_out in result:
            ## 티켓이 다 팔렸음.
            theater['num_renegers'][movie]+=1
            ## 프로세스 종료 
            env.exit()
        elif req in result:
            if num_tickets > theater['available'][movie]:
                ## 티켓이 있는데 부족할 경우에는 그냥 감 
                print("{:7.1f}: we don't have enough tickets".format(env.now))
                yield env.timeout(0.5)
                env.exit()
            else:
                theater['available'][movie] -= num_tickets
                if theater['available'][movie] < 1:
                    ## 아래 보면 이벤트를 succeed로 변경함. 
                    ## 따라서 만약 다른 프로세스에서 yield req | theater['sold_out'][movie] 등으로 참고하고 있을 경우 
                    ## 아래 부분이 수행되는 즉시, yield 구문을 빠져오게 됨. 
                    theater['sold_out'][movie].succeed()
                    theater['when_sold_out'][movie] = env.now
                    theater['available'][movie] = 0
                    print('{} is sold out'.format(movie))
                yield env.timeout(1)
    
def customer_arrivals(env, theater):
    ## exponential time마다 사람이 도착함 
    ## 영화, 사람 수 등을 랜덤하게 고르고, 티켓이 남아있을 경우 이를 프로세스로 env에 넘겨줌
    while True:
        yield env.timeout(np.random.exponential(2))
        movie = np.random.choice(theater['movies'])
        num_tickets = np.random.randint(1, 6)
        ## 티켓이 남아지 않으면 새로운 사람이 프로세스로 넘어가지 않는다. 
        if theater['available'][movie]!=0:
            print("{:7.1f}: {} customer arrived to see {}".format(env.now, num_tickets, movie))
            env.process(movie_goer(env, movie, num_tickets, theater))

np.random.seed(42)
env = simpy.Environment()

print("Movie renege")
## 이전에는 모두 class, process등으로 처리했는데 여기서는 딕셔너리로 데이터를 관리함. 
## 딕셔너리에 Resource, event 등이 포함되어 있음. 
## 경우에 따라 클래스로 만들지 않고, 아래처럼 딕셔너리로 관리하는 것이 편할 수도 있음. 
movies = ['Die hard 2', "Kill bill", 'Resevoir Dogs']
theater = {
    'counter' : simpy.Resource(env, capacity=1),
    'movies' : movies, 
    'available' : {movie: 20 for movie in movies}, 
    'sold_out' : {movie: env.event() for movie in movies}, 
    'when_sold_out' : {movie: None for movie in movies}, 
    'num_renegers' : {movie: 0 for movie in movies}
}

env.process(customer_arrivals(env, theater))
env.run(until=100)

Movie renege
    0.9: 3 customer arrived to see Die hard 2
    4.0: 2 customer arrived to see Die hard 2
    4.3: 3 customer arrived to see Resevoir Dogs
    8.3: 5 customer arrived to see Resevoir Dogs
   15.3: 2 customer arrived to see Kill bill
   15.7: 1 customer arrived to see Die hard 2
   16.5: 5 customer arrived to see Kill bill
   17.6: 3 customer arrived to see Die hard 2
   19.5: 4 customer arrived to see Kill bill
   26.8: 3 customer arrived to see Resevoir Dogs
   27.7: 3 customer arrived to see Die hard 2
   31.7: 5 customer arrived to see Resevoir Dogs
   32.9: 4 customer arrived to see Kill bill
   38.6: 2 customer arrived to see Kill bill
   39.5: 5 customer arrived to see Kill bill
   39.6: we don't have enough tickets
   40.1: 4 customer arrived to see Resevoir Dogs
Resevoir Dogs is sold out
   40.3: 3 customer arrived to see Die hard 2
   45.1: 4 customer arrived to see Kill bill
   45.1: we don't have enough tickets
   45.9: 2 customer arrived to see Kill bill
   4

In [500]:
import simpy 
import numpy as np 

def simple_process(env, shared_event):
    while True:
        result = yield env.timeout(10.0) | shared_event
        if shared_event in result:
            print("shared event occurs at {}".format(env.now))
            break
        print("shared event didn't occurs during {}".format(env.now))

def global_process(env, shared_event):
    while True: 
        yield env.timeout(1)
        if np.random.randint(0, 50)==1:
            shared_event.succeed()
            break

np.random.seed(21)

env = simpy.Environment()
shared_event = env.event()

env.process(simple_process(env, shared_event))
env.process(global_process(env, shared_event))

env.run(until=100)

shared event didn't occurs during 10.0
shared event didn't occurs during 20.0
shared event didn't occurs during 30.0
shared event didn't occurs during 40.0
shared event didn't occurs during 50.0
shared event didn't occurs during 60.0
shared event didn't occurs during 70.0
shared event didn't occurs during 80.0
shared event didn't occurs during 90.0
shared event occurs at 91


In [541]:
import simpy 
import numpy as np 

np.random.seed(42)

def people(env, p_num, water_pump, water_container):
    with water_pump.request() as req:
        yield req
        required_water = np.random.uniform(10, 100)
        if water_container.level >= required_water:
            water_container.get(required_water)
            yield env.timeout(required_water/10)
            print("people {:2d} used {:6.2f} fuel".format(p_num, required_water))
        else: 
            print("water container can't refuel it")
        print("{:6.2f}: water {:8.2f} remained".format(env.now, water_container.level))
    
def people_arrival(env,water_pump, water_container):
    for p_num in (i for i in range(0, 1000)):
        arrival_time = np.random.triangular(left=1, right=5, mode=1)
        print("{:6.2f}: people {:2d} arrived".format(env.now, p_num))
        yield env.timeout(arrival_time)
        env.process(people(env, p_num, water_pump, water_container))
        
def water_refueling(env, water_container):
    while True:
        if (water_container.level / water_container.capacity) <= 0.1:
            refuel_amount = water_container.capacity - water_container.level 
            ## 1초에 50씩 넣음.
            for i in range(0, int(refuel_amount//50)):
                yield env.timeout(1)
                water_container.put(50)
            yield env.timeout(1)
            water_container.put(refuel_amount%50)
            print("{:6.2f}: water_contaienr refueled".format(env.now))
        ## 5 초에 한번씩 현재 물 양을 체크함
        yield env.timeout(5)
            
        
env = simpy.Environment()
water_pump = simpy.Resource(env, capacity=1)
water_reservoir = simpy.Container(env, capacity = 500, init=10)

env.process( people_arrival(env, water_pump, water_reservoir) )
env.process(water_refueling(env, water_reservoir))

env.run(until=20)



  0.00: people  0 arrived
  1.84: people  1 arrived
water container can't refuel it
  1.84: water    60.00 remained
  5.95: people  2 arrived
people  1 used  24.04 fuel
  8.35: water   385.96 remained
  8.41: people  3 arrived
  9.74: people  4 arrived
people  2 used  15.23 fuel
  9.94: water   420.73 remained
 10.00: water_contaienr refueled
 13.28: people  5 arrived
 16.12: people  6 arrived
people  3 used  64.10 fuel
 16.35: water   396.63 remained
 17.16: people  7 arrived


In [578]:
import simpy 
import numpy as np 

np.random.seed(42)

class Gas_Station(object):
    def __init__(self, env):
        ## 환경 세팅, 리소스/컨테이너 생성, 함께 시작되어야 하는 프로세스 추가 
        self.env = env
        self.gas_pump = simpy.Resource(self.env, capacity=2)
        self.gas_container = simpy.Container(self.env, capacity=500, init=10)
        self.env.process(self.gas_checking())
        self.is_refueling = False
    
    def gas_fueling(self, c_num, required_amount):
        ## 차가 오면 가스를 채워주는 프로세스 
        with self.gas_pump.request() as req:
            yield req
            if required_amount > self.gas_container.level:
                ## 가스를 채우려고 보니 가스가 없음
                print("{:8.2f}: {:6.2f} fuel remained".format(self.env.now, self.gas_container.level))
                print("{:8.2f}: car{:2d} leaved".format(self.env.now,c_num))
                self.is_refueling = True
                yield self.env.process(self.gas_refueling())
                self.is_refueling = False
            else:
                ## 가스가 있음.
                self.gas_container.get(required_amount)
                yield self.env.timeout(required_amount/10)
                print("{:8.2f}: car{:2d} used {:6.2f} fuel".format(self.env.now, c_num, required_amount))
                print("{:8.2f}: {:6.2f} fuel remained".format(self.env.now, self.gas_container.level))
    
    def gas_refueling(self):
        ## 가스가 없을때 가스를 채우는 프로세스 
        refuel_amount = self.gas_container.capacity - self.gas_container.level
        print("{:8.2f}: gas_container refuel start".format(self.env.now))
        for i in range(0, int(refuel_amount//50)):
            yield self.env.timeout(1)
            self.gas_container.put(50)
        print("{:8.2f}: gas_container refueled".format(self.env.now))
        
    def gas_checking(self):
        ## 가스가 일정 값 이상으로 떨어지지 않았는지 확인하는 프로세스 
        while True:
            if self.gas_container.level/ self.gas_container.capacity <= 0.1:
                ## 차가 왔을 때 이미 가스가 부족한 상황이면, 위쪽에서 이 프로세스를 돌리고 있을 수 있음. 
                ## 따라서 self.is_refueling등으로 제어하지 않으면 동시에 프로세스가 돌아가고 있을 수 있으므로
                ## 이미 실행중일 때는 실행되지 않도록 제어한다. 
                if self.is_refueling != True:
                    yield self.env.process(self.gas_refueling())
            yield env.timeout(5)

def car_generator(env, gas_station):
    ## 일정 시간 별로 차에 가스 채우는 프로세스를 수행하는 제너레이터 
    for i in range(0, 1000):
        arrival_time = np.random.exponential(5)
        required_amount = np.random.uniform(10, 100)
        yield env.timeout(arrival_time)
        print("{:8.2f}: car{:2d} arrived".format(env.now, i))
        env.process(gas_station.gas_fueling(i, required_amount))
        
env = simpy.Environment()

gas_station = Gas_Station(env)
env.process(car_generator(env, gas_station))

env.run(until=50)

    0.00: gas_container refuel start
    2.35: car 0 arrived
    8.93: car 1 arrived
    9.00: gas_container refueled
    9.78: car 2 arrived
   10.08: car 3 arrived
   11.90: car 0 used  95.56 fuel
   11.90: 300.56 fuel remained
   14.31: car 2 used  24.04 fuel
   14.31: 276.52 fuel remained
   14.67: car 4 arrived
   14.78: car 5 arrived
   15.32: car 1 used  63.88 fuel
   15.32: 188.56 fuel remained
   22.69: car 4 used  73.73 fuel
   22.69: 114.83 fuel remained
   23.10: car 3 used  87.96 fuel
   23.10:  17.54 fuel remained
   23.71: car 6 arrived
   23.71:  17.54 fuel remained
   23.71: car 6 leaved
   23.71: gas_container refuel start
   24.71: car 7 arrived
   26.53: car 8 arrived
   29.35: car 9 arrived
   32.42: car 5 used  97.29 fuel
   32.42: 417.54 fuel remained
   32.71: gas_container refueled
   34.09: car10 arrived
   35.07: car 7 used  26.51 fuel
   35.07: 383.81 fuel remained
   35.81: car11 arrived
   38.43: car 8 used  57.23 fuel
   38.43: 347.60 fuel remained
   38.

In [544]:
gas_station = Gas_Station(env)

TypeError: 'module' object is not callable

In [623]:
## *args: non-keyworded variable-length argument list 
## **kwargs: keyworded, variable-length argument dictionary

## argument에는 *를 붙이고, 그냥 사용할 때는 * 없이 사용합니다. 
def test_args(*args, **kwargs):
    print(type(kwargs))
    print(type(args))## tuple
    print("-"*20)
    for i, x in enumerate(args):
        print("{}: {}".format(i, x))
    print("#"*20)
test_args(1,2)
test_args(1,2,3)
test_args(1,'a', lambda x: x+1, [1,2,3])

<class 'dict'>
<class 'tuple'>
--------------------
0: 1
1: 2
####################
<class 'dict'>
<class 'tuple'>
--------------------
0: 1
1: 2
2: 3
####################
<class 'dict'>
<class 'tuple'>
--------------------
0: 1
1: a
2: <function <lambda> at 0x11c3ba730>
3: [1, 2, 3]
####################


In [625]:
def func1(a, b):
    return a+b

print(func1(1, 2))
print(func1(1, 2, 3))

3


TypeError: func1() takes 2 positional arguments but 3 were given

In [631]:
## 함수 선언부에는 *args 로 써주고 
## 값을 사용할 때는 args 로 사용합니다. 
def func1(*args):
    print(type(args))
    print(sum(args))
    print("="*20)

func1(1, 2)
func1(1, 2, 3)
func1(1, 2, 3, 4)

<class 'tuple'>
3
<class 'tuple'>
6
<class 'tuple'>
10


In [639]:
def func2(**kwargs):
    print(type(kwargs))
    for k in kwargs.keys():
        print("{}: {}".format(k, kwargs[k]))
    print("="*20)

func2(name='Lee', age='25')
func2(name='Kim', age='25', gpa=4.3)

<class 'dict'>
name: Lee
age: 25
<class 'dict'>
name: Kim
age: 25
gpa: 4.3


In [660]:
## hybrid
def func_hybrid(*args, **kwargs):
    print("args")
    for i, v in enumerate(args):
        print("{}: {}".format(i, v))
    print("-"*20)
    for k, v in kwargs.items():
        print("{}: {}".format(k, v))
    print("="*20)
func_hybrid(1,2,3, lambda x:x+10, name='Lee', age=25, gpa=4.3, f = lambda x: x+1)

args
0: 1
1: 2
2: 3
3: <function <lambda> at 0x10fb0dd08>
--------------------
name: Lee
age: 25
gpa: 4.3
f: <function <lambda> at 0x11c3ba950>


In [654]:
func_hybrid(1,2,3, name='Lee', 4)

SyntaxError: positional argument follows keyword argument (<ipython-input-654-87dd3f44c5c2>, line 1)

In [667]:
def func4(*args):
    print(args)
    print(*args)
    print("="*20)
    def temp_func(*aa):
        print(aa)
    temp_func(args, 1)
    temp_func(*args)
func4(1,2,3)

(1, 2, 3)
1 2 3
((1, 2, 3), 1)
(1, 2, 3)


In [664]:
## 이름이 똑같지 않아도 됩니다. 
def func_different_name(*input_tuple, **input_dict):
    for i, v in enumerate(input_tuple):
        print("{:2d}: {}".format(i, v))
    print("=== args over ===")
    for k, v in input_dict.items():
        print("{}: {}".format(k, v))
    print("=== kwargs over ===")
func_different_name(1,2,3, name='lee', age=35)

 0: 1
 1: 2
 2: 3
=== args over ===
name: lee
age: 35
=== kwargs over ===


In [666]:
def func_different_name(*input_tuple, **input_dict, ):
    for i, v in enumerate(input_tuple):
        print("{:2d}: {}".format(i, v))
    print("=== args over ===")
    for k, v in input_dict.items():
        print("{}: {}".format(k, v))
    print("=== kwargs over ===")
func_different_name(1,2,3, name='lee', age=35)

SyntaxError: invalid syntax (<ipython-input-666-83b0005cb13b>, line 1)

In [680]:
def func4(*args):
    def print_func(*args):
        for i, v in enumerate(args):
            print("{}: {}".format(i, v))
        print("#"*20)
    ## 아래의 경우 단 하나의 argument가 넘어가는 것이고 
    print_func(args)
    ## 아래의 경우는 여러 개의 argument가 넘어오는 것 
    print_func(*args)
func4(1,2,3)

0: (1, 2, 3)
####################
0: 1
1: 2
2: 3
####################


In [711]:
import numpy as np 
import simpy 

def producer(env, store):
    ## 무한으로 진행되는 generator
    ## 1초에 제품을 하나씩 생성해냄. 
    i = 1
    while True:
        yield env.timeout(1)
        product_name = "spam{:2d}".format(i)
        ## store method: items, put, get 
        yield store.put(product_name)
        ## 현재 창고에 있는 제품들 출력 
        print("{:6.2f}: Produced {}, items: {}".format(env.now, product_name, store.items))
        i+=1

def consumer(name, env, store):
    ## 고객 한명이 스팸을 요구하고, 
    ## 한번 사간 다음에는 한동안 안 사감 
    while True:
        yield env.timeout(1)
        print("{:6.2f}: {} requesting spam".format(env.now, name))
        ## 아래도 request와 마찬가지로 획득할때까지 기다렸다가 생기면 가져감 
        waiting_time = env.now
        item = yield store.get()
        waiting_time = env.now - waiting_time
        #print("{} was waiting during {:6.2f}".format(name, waiting_time))
        print("{:6.2f}: {} got {}".format(env.now, name, item))
        ## 물건을 사갔으니 한동안 안 사감 
        yield env.timeout(np.random.exponential(5))

def consumers(env, store, n):
    ## 총 n 명의 고객을 프로세스로 등록함  
    for i in range(0, n):
        yield env.timeout(2)
        new_consumer = consumer("consumer{:2d}".format(i), env, store)
        env.process(new_consumer)

np.random.seed(42)
env = simpy.Environment()
store = simpy.Store(env, capacity=2)

prod = env.process(producer(env, store))
#consumers = [env.process(consumer("consumer{:2d}".format(i), env, store)) for i in range(0, 2)]
env.process(consumers(env, store, 5))

env.run(until=10)

  1.00: Produced spam 1, items: ['spam 1']
  2.00: Produced spam 2, items: ['spam 1', 'spam 2']
  3.00: consumer 0 requesting spam
  3.00: consumer 0 got spam 1
  3.00: Produced spam 3, items: ['spam 2', 'spam 3']
  5.00: consumer 1 requesting spam
  5.00: consumer 1 got spam 2
  5.00: Produced spam 4, items: ['spam 3', 'spam 4']
  6.35: consumer 0 requesting spam
  6.35: consumer 0 got spam 3
  6.35: Produced spam 5, items: ['spam 4', 'spam 5']
  7.00: consumer 2 requesting spam
  7.00: consumer 2 got spam 4
  7.35: Produced spam 6, items: ['spam 5', 'spam 6']
  9.00: consumer 3 requesting spam
  9.00: consumer 3 got spam 5
  9.00: Produced spam 7, items: ['spam 6', 'spam 7']


In [836]:
import simpy 
import numpy as np 

skill_set = ["skill{:2d}".format(i) for i in range(0, 3)]

class Human(object):
    def __init__(self, name):
        self.name = name
        ## 몇 개의 skill을 가지게 될지 정함 
        self.skill_num = np.random.randint(1, 3)
        ## 가지고 있는 skill은 중복될 수 없으며, 따라서 set로 관리해야 편한
        ## 특히, 이렇게 해야 skill set이 subset인지 아닌지를 더 잘 알 수 있음. 
        self.skills = set(np.random.choice(skill_set, self.skill_num, replace=False))
    def __repr__(self):
        ## 프린트할때는 여기서 리턴되는 스트링을 참고하여 만들어짐 
        return "{}".format(self.skills)
        #return "{} have skills: {}\n".format(self.name, self.skills)
        
def task(env, name, human_resources):
    skill_num = np.random.randint(1, 3)## 해당 태스크는 몇 가지의 스킬을 필요로 하는가 
    required_skills = set(np.random.choice(skill_set, skill_num, replace=False))
    ## 현재 사용가능한 사람 수 
    print("{:6.2f}: available humans count: {}".format(env.now, len(human_resources.items)))
    ## 태스크 대기열 수 
    print("{:6.2f}: task queue count: {}".format(env.now, len(human_resources.get_queue)))
    ## 해당 태스크가 필요로 하는 스킬 
    print("{:6.2f}: {} required skills: {}".format(env.now, name, required_skills))
    ## 사용하는 시간동안은 뺐다가, 사용이 끝나면 다시 store에 넣어줌 
    ## 아래처럼 원하는 조건을 넘기면, 그 조건에 맞는 resource를 찾아서 정리해줌. 
    ## 여러 가지의 리소스가 있을때는 어떤 것이 되는지 모르겠지만, 어떤 것이 되어도 상관없지 않은가.
    human = yield human_resources.get(lambda x: required_skills.issubset(x.skills))
    print("{:6.2f}: {} start to handle {}".format(env.now, human.name, name))
    yield env.timeout(30)
    print("{:6.2f}: {} completed".format(env.now, name))
    yield human_resources.put(human)
    

def task_generator(env, human_resources):
    for i in range(0, 100):
        yield env.timeout(3)
        env.process(task(env, "task{:2d}".format(i), human_resources))
        
np.random.seed(42)
env = simpy.Environment()

human_resources = simpy.FilterStore(env)## capacity를 명시하지 않으면 inf로 인식함. 즉 존나 마음껏 할수 있다는 이야기. 
human_resources.items = [Human("human{:2d}".format(i)) for i in range(0, 5)]

env.process( task_generator(env, human_resources) )
env.run(until=20)

  3.00: available humans count: 5
  3.00: task queue count: 0
  3.00: task 0 required skills: {'skill 0', 'skill 1'}
  6.00: available humans count: 5
  6.00: task queue count: 1
  6.00: task 1 required skills: {'skill 0'}
  9.00: available humans count: 5
  9.00: task queue count: 2
  9.00: task 2 required skills: {'skill 0', 'skill 2'}
 12.00: available humans count: 5
 12.00: task queue count: 3
 12.00: task 3 required skills: {'skill 1', 'skill 2'}
 12.00: human 4 start to handle task 3
 15.00: available humans count: 4
 15.00: task queue count: 3
 15.00: task 4 required skills: {'skill 0', 'skill 2'}
 18.00: available humans count: 4
 18.00: task queue count: 4
 18.00: task 5 required skills: {'skill 1'}
 18.00: human 0 start to handle task 5


In [814]:
('skill 1', 'skill 2') in ('skill 2', 'skill 1', 'skill 3')

False

In [713]:
## random multiple choice

import numpy as np 

cands = [i for i in range(0, 10)]
np.random.choice(cands, 10, replace=False)

array([4, 2, 0, 8, 9, 6, 5, 3, 7, 1])

In [724]:
import numpy as np 

cands = [i for i in range(0, 10)]

def selection(basket, n):
    new_basket = basket.copy()
    r_lst = []
    for i in range(0, n):
        temp = new_basket.pop(np.random.choice([i for i in range(0, len(new_basket))]))
        r_lst.append(temp)
    return r_lst

print( selection(cands, 10) )

[8, 6, 1, 4, 0, 7, 3, 5, 9, 2]


In [736]:
np.random.randint(1, 3, 10)

array([2, 1, 1, 1, 2, 1, 1, 1, 2, 1])

In [894]:
import simpy 
import numpy as np 

def Producer(_env, _PStore):
    ## 최대 100개의 제품을 0.2초마다 만들어내는 제너레이터 
    ## 가장 먼저 만들어진 제품에 우선순위를 높게 둬서 빠르게 내보내는 작업을 수행 
    for i in range(0, 1000):
        yield _env.timeout(0.2)
        new_item = simpy.PriorityItem(priority=round(_env.now, 2), item="PDT{:2d}".format(i))
        yield _PStore.put(new_item)

def consumer(_env, _name, _PStore):
    ## 도착 ==> 제품 획득 => 2초 기다림 ==> 출력 및 종료 
    print("{:6.2f}: {} arrived".format(_env.now, _name))
    #print("{:6.2f}: {} items remained".format(_env.now, _PStore.items))
    get_item = yield _PStore.get()
    yield _env.timeout(2)
    print("{:6.2f}: {} get {}".format(_env.now, _name, get_item))
    
def consumers(_env, _PStore):
    ## 일정 시간 간격으로 고객을 도착
    ## 도착 시간 간격은 exponential time, 사람 수는 poisson dist
    i = 0
    while True:
        ## 도착 시간은 exponential time
        yield env.timeout(np.random.exponential(3))
        ## 도착하는 사람 수는 poisson dist 
        customers_count = np.random.poisson(3)
        if customers_count != 0:
            print("{:6.2f}: {} customer arrived".format(_env.now, customers_count))
            for j in range(0, customers_count):
                ## 아래처럼 포맷팅을 할 경우 zero-padding이 됨 
                _env.process(consumer(_env, "consumer({:0>2d}_{:0>2d})".format(i, j), _PStore))
            i+=1

np.random.seed(42)
env = simpy.Environment()
PStore = simpy.PriorityStore(env) ## capacity => inf

env.process(Producer(env, PStore))
env.process(consumers(env, PStore))

env.run(until=20)

  1.41: 4 customer arrived
  1.41: consumer(00_00) arrived
  1.41: consumer(00_01) arrived
  1.41: consumer(00_02) arrived
  1.41: consumer(00_03) arrived
  1.59: 3 customer arrived
  1.59: consumer(01_00) arrived
  1.59: consumer(01_01) arrived
  1.59: consumer(01_02) arrived
  3.41: consumer(00_00) get PriorityItem(priority=0.2, item='PDT 0')
  3.41: consumer(00_01) get PriorityItem(priority=0.4, item='PDT 1')
  3.41: consumer(00_02) get PriorityItem(priority=0.6, item='PDT 2')
  3.41: consumer(00_03) get PriorityItem(priority=0.8, item='PDT 3')
  3.59: consumer(01_00) get PriorityItem(priority=1.0, item='PDT 4')
  3.59: consumer(01_01) get PriorityItem(priority=1.2, item='PDT 5')
  3.59: consumer(01_02) get PriorityItem(priority=1.4, item='PDT 6')
 12.10: 2 customer arrived
 12.10: consumer(02_00) arrived
 12.10: consumer(02_01) arrived
 12.71: 3 customer arrived
 12.71: consumer(03_00) arrived
 12.71: consumer(03_01) arrived
 12.71: consumer(03_02) arrived
 14.10: consumer(02_00) g

In [873]:
round(0.6666666, 2)

0.67

In [881]:
for i in range(0, np.infty):
    if i==10:
        break
    print(i)

TypeError: 'float' object cannot be interpreted as an integer

In [889]:
for i in 10:
    print(i)

TypeError: 'int' object is not iterable

/Users/frhyme/anaconda3/lib/python3.6/site-packages/numpy/core/function_base.py:128: RuntimeWarning: invalid value encountered in multiply
  y = y * step


array([ nan,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf])